## ClickHouse для аналитики

## Установка ClickHouse

ClickHouse можно установить на Linux (подробно про установку написано в документации https://clickhouse.yandex/).

На Windows или Mac можно запустить ClickHouse под Docker'ом: первая команда поднимает clickhouse-server на порту 8123, а вторая - позволяет подключиться к консольному ClickHouse клиенту.

```docker run -d --name clickhouse-server --publish=8123:8123 --publish=9000:9000 yandex/clickhouse-server
docker run -it --rm --link clickhouse-server:9000 yandex/clickhouse-client --host clickhouse-server```


## Загружаем данные

У Метрики и AppMetrica есть функциональность Logs API, которая позволяет получить сырые данные по пользовательским просмотрам страниц и сессиям. Мы для примера будем использовать данные с демо-счетчика, который стоит на сайт Help'a Яндекс.Метрики и доступен всем желающим.

**Полезные ссылки**
* [Logs API AppMetrica](https://tech.yandex.ru/appmetrica/doc/mobile-api/logs/about-docpage/)
* [Logs API Метрика](https://tech.yandex.ru/metrika/doc/api2/logs/intro-docpage/)
* [Скрипт для интеграции Logs API Метрики с ClickHouse](https://tech.yandex.ru/metrika/doc/api2/logs/clickhouse-integration-docpage/)
* [Framework для интеграции Logs API AppMetrica c ClickHouse]()

In [18]:
import requests
import pandas as pd
import io
import os
import time
pd.set_option('display.max_colwidth', 5000)


from plotly import __version__
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
from plotly import graph_objs as go
import plotly

init_notebook_mode(connected = True)



def plotly_line_df(df, title = '', filename = None):
    data = []
    
    for column in df.columns:
        trace = go.Scatter(
            x = df.index,
            y = df[column],
            mode = 'lines',
            name = column
        )
        data.append(trace)
    
    layout = dict(title = title)
    fig = dict(data = data, layout = layout)
    iplot(fig, show_link = False)
    if filename is not None:
        plotly.offline.plot(fig, show_link = False, filename = filename)
        
def plotly_bar_df(df, title = '', filename = None, index = None):
    data = []
    
    if index is None:
        for column in df.columns:
            trace = go.Bar(
                x = df.index,
                y = df[column],
                name = column
            )
            data.append(trace)
    else:
        for column in df.columns:
            trace = go.Bar(
                x = index,
                y = df.loc[index][column],
                name = column
            )
            data.append(trace)
    
    layout = dict(title = title)
    fig = dict(data = data, layout = layout)
    iplot(fig, show_link = False)
    if filename is not None:
        plotly.offline.plot(fig, show_link = False, filename = filename)

In [19]:
visits_tmp_dfs = []

for filename in os.listdir('demo_counter_data'):
    if filename.startswith('visits'):
        visits_tmp_dfs.append(pd.read_csv('./demo_counter_data/' + filename, sep = '\t'))

/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2698: DtypeWarning:

Columns (25) have mixed types. Specify dtype option on import or set low_memory=False.

/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2698: DtypeWarning:

Columns (16) have mixed types. Specify dtype option on import or set low_memory=False.

/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2698: DtypeWarning:

Columns (31) have mixed types. Specify dtype option on import or set low_memory=False.



In [20]:
visits_df = pd.concat(visits_tmp_dfs)

In [21]:
visits_columns = [
    'ym:s:bounce',
    'ym:s:browser',
    'ym:s:browserMajorVersion',
    'ym:s:browserMinorVersion',
    'ym:s:clientID',
    'ym:s:date',
    'ym:s:dateTime',
    'ym:s:goalsID',
    'ym:s:deviceCategory',
    'ym:s:goalsDateTime',
    'ym:s:ipAddress',
    'ym:s:isNewUser',
    'ym:s:lastAdvEngine',
    'ym:s:lastReferalSource',
    'ym:s:lastSearchEngine',
    'ym:s:lastSearchEngineRoot',
    'ym:s:lastSocialNetwork',
    'ym:s:lastSocialNetworkProfile',
    'ym:s:lastTrafficSource',
    'ym:s:mobilePhone',
    'ym:s:mobilePhoneModel',
    'ym:s:operatingSystem',
    'ym:s:operatingSystemRoot',
    'ym:s:pageViews',
    'ym:s:params',
    'ym:s:referer',
    'ym:s:regionCity',
    'ym:s:regionCountry',
    'ym:s:screenHeight',
    'ym:s:screenWidth',
    'ym:s:startURL',
    'ym:s:endURL',
    'ym:s:visitDuration',
    'ym:s:visitID',
    'ym:s:watchIDs'
]

ch_types = {
    'ym:s:bounce': 'UInt8',
    'ym:s:browser': 'String',
    'ym:s:browserMajorVersion': 'UInt16',
    'ym:s:browserMinorVersion': 'UInt16',
    'ym:s:clientID': 'UInt64',
    'ym:s:date': 'Date',
    'ym:s:dateTime': 'DateTime',
    'ym:s:deviceCategory': 'String',
    'ym:s:endURL': 'String',
    'ym:s:goalsDateTime': 'Array(DateTime)',
    'ym:s:goalsID': 'Array(UInt32)',
    'ym:s:ipAddress': 'String',
    'ym:s:isNewUser': 'UInt8',
    'ym:s:lastAdvEngine': 'String',
    'ym:s:lastReferalSource': 'String',
    'ym:s:lastSearchEngine': 'String',
    'ym:s:lastSearchEngineRoot': 'String',
    'ym:s:lastSocialNetwork': 'String',
    'ym:s:lastSocialNetworkProfile': 'String',
    'ym:s:lastTrafficSource': 'String',
    'ym:s:mobilePhone': 'String',
    'ym:s:mobilePhoneModel': 'String',
    'ym:s:operatingSystem': 'String',
    'ym:s:operatingSystemRoot': 'String',
    'ym:s:pageViews': 'Int32',
    'ym:s:params': 'Array(String)',
    'ym:s:referer': 'String',
    'ym:s:regionCity': 'String',
    'ym:s:regionCountry': 'String',
    'ym:s:screenHeight': 'UInt16',
    'ym:s:screenWidth': 'UInt16',
    'ym:s:startURL': 'String',
    'ym:s:visitDuration': 'UInt32',
    'ym:s:visitID': 'UInt64',
    'ym:s:watchIDs': 'Array(UInt64)'
}

In [14]:
# visits_df.VisitID.head(10)

In [22]:
columns = []

for param in visits_columns:
    param_name = param.replace('ym:s:', '')[0].capitalize() + param.replace('ym:s:', '')[1:]
    param_type = ch_types[param]
    columns.append('%s %s' % (param_name, param_type))

На этот раз создадим таблицу с ENGINE MergeTree. В ClickHouse есть целое семейство движков MergeTree - Collapsing, Summing, Replacing, Replicated и т.д.
MergeTree - самая простая версия, которая поддерживает индексирование и семплирование. 

In [23]:
HOST = 'http://localhost:8123/'
def get_clickhouse_data(query, host = HOST, connection_timeout = 1500):
    NUMBER_OF_TRIES = 30
    DELAY = 10
    
    for i in range(NUMBER_OF_TRIES):
        r = requests.post(host, params = {'timeout_before_checking_execution_speed': 120, 'max_execution_time': 6000}
                          , timeout = connection_timeout, data = query)
        if r.status_code == 200:
            return r.text
        else:
            print('ATTENTION: try #%d failed' % i)
            if i != (NUMBER_OF_TRIES-1):
                print(query)
                print(r.text)
                time.sleep(DELAY*(i+1))
            else:
                raise ValueError(r.text)

In [24]:
q = 'DROP TABLE IF EXISTS visits_all'
get_clickhouse_data(q)

q = '''
    CREATE TABLE visits_all ({columns}) 
    ENGINE = MergeTree(Date, intHash32(ClientID), (Date, intHash32(ClientID)), 8192)
'''.format(columns = ', '.join(columns))

get_clickhouse_data(q)

''

In [27]:
visits_df['ym:s:browserMinorVersion'] = list(map(int, list(visits_df['ym:s:browserMinorVersion'].fillna(0))))
visits_df['ym:s:visitID'] = list(map(int, list(visits_df['ym:s:visitID'].fillna(0))))

In [28]:
visits_df = visits_df.fillna('')

In [29]:
visits_df.shape

(1222286, 57)

In [30]:
visits_df = visits_df[visits_columns]

In [31]:
%%time
values = []
values.append('\t'.join(visits_columns))

for i in range(visits_df.shape[0]):
    if i % 100000 == 0:
        print(i)
    values.append('\t'.join(map(str, visits_df.iloc[i].values)))

0
100000
200000
300000
400000
500000
600000
700000
800000
900000
1000000
1100000
1200000
CPU times: user 4min 58s, sys: 4.21 s, total: 5min 2s
Wall time: 5min 13s


In [32]:
HOST = 'http://localhost:8123/'
def upload(table, content, host=HOST):
    '''Uploads data to table in ClickHous'''
    content = content.encode('utf-8')
    query_dict = {
             'query': 'INSERT INTO ' + table + ' FORMAT TabSeparatedWithNames '
        }
    r = requests.post(host, data=content, params=query_dict)
    result = r.text
    if r.status_code == 200:
        return result
    else:
        raise ValueError(r.text)

In [33]:
%%time
upload('visits_all', '\n'.join(values))

CPU times: user 4.6 s, sys: 17.2 s, total: 21.8 s
Wall time: 1min 56s


''

## Простые SELECT запросы

Для проверки попробуем сделать простой SELECT нескольких столбцов.

ClickHouse - столбцовая СУБД, то есть, данные хранятся по колонкам и сжатие так же происходит по столбцам. Типичный сценарий работы с ClickHouse - в запросе участвует небольшое количество колонок, но относительно большое количество строк. Поэтому делать `SELECT *` не рекомендуется.

In [34]:
q = '''
    SELECT 
        ClientID,
        DateTime,
        StartURL, 
        Params
    FROM visits_all
    LIMIT 10
'''

print(get_clickhouse_data(q))

1461908033834334420	2016-10-01 16:17:26	https://yandex.ru/support/metrika/troubleshooting/access-qanda.xml	['{"login":true}','{"login":true}']
1469802107825757398	2016-10-01 23:33:48	https://yandex.ru/support/metrika/	[]
1474720846487102126	2016-10-01 06:32:42	https://yandex.ru/support/metrika/data/e-commerce.xml	[]
1472198459221414089	2016-10-01 20:35:32	https://yandex.ru/support/metrika/troubleshooting.xml	['{"login":true}','{"login":true}']
1467057414858508354	2016-10-01 00:30:19	https://yandex.ru/support/metrika/general/goals.xml	[]
1475350611504457143	2016-10-01 23:52:39	https://yandex.ru/support/metrika/general/access.xml	[]
1466280569981691679	2016-10-01 00:21:18	https://yandex.ru/support/metrika/reports/add-goals.xml	[]
1474974033103072669	2016-10-01 09:58:06	https://yandex.ru/support/metrika/general/glossary.xml	[]
1475329004492101252	2016-10-01 16:37:01	https://yandex.ru/support/metrika/	[]
147261681766100953	2016-10-01 17:36:57	https://yandex.ru/support/metrika/sources/sourc

Давайте попробуем сделать вывод более пригодным для работы и преобразуем вывод ClickHouse в `pandas.DataFrame`.

In [35]:
def get_clickhouse_df(query, host = HOST, connection_timeout = 1500):
    data = get_clickhouse_data(query, host, connection_timeout) 
    df = pd.read_csv(io.StringIO(data), sep = '\t')
    return df

На самом деле, можно сделать еще удобнее, указав `FORMAT TabSeparatedWithNames` -  тогда в ответе придет также строке headers, которая будет распаршена как название колонок `DataFrame`.

In [36]:
q = '''
    SELECT 
        ClientID,
        DateTime,
        StartURL, 
        Params
    FROM visits_all
    LIMIT 10
    FORMAT TabSeparatedWithNames
'''

get_clickhouse_df(q)

,ClientID,DateTime,StartURL,Params
0,1461908033834334420,2016-10-01 16:17:26,https://yandex.ru/support/metrika/troubleshooting/access-qanda.xml,"['{""login"":true}','{""login"":true}']"
1,1469802107825757398,2016-10-01 23:33:48,https://yandex.ru/support/metrika/,[]
2,1474720846487102126,2016-10-01 06:32:42,https://yandex.ru/support/metrika/data/e-commerce.xml,[]
3,1472198459221414089,2016-10-01 20:35:32,https://yandex.ru/support/metrika/troubleshooting.xml,"['{""login"":true}','{""login"":true}']"
4,1467057414858508354,2016-10-01 00:30:19,https://yandex.ru/support/metrika/general/goals.xml,[]
5,1475350611504457143,2016-10-01 23:52:39,https://yandex.ru/support/metrika/general/access.xml,[]
6,1466280569981691679,2016-10-01 00:21:18,https://yandex.ru/support/metrika/reports/add-goals.xml,[]
7,1474974033103072669,2016-10-01 09:58:06,https://yandex.ru/support/metrika/general/glossary.xml,[]
8,1475329004492101252,2016-10-01 16:37:01,https://yandex.ru/support/metrika/,[]
9,147261681766100953,2016-10-01 17:36:57,https://yandex.ru/support/metrika/sources/sources-summary.xml,[]


Как и в стандартном SQL, в ClickHouse можно указать порядок вывода с помощью `ORDER BY` и число строк, которые должны быть в выводе с помощью `LIMIT`.

Также в секции `WHERE` можно указать дополнительные фильтры.

In [37]:
q = '''
    SELECT 
        ClientID,
        DateTime,
        StartURL, 
        Params
    FROM visits_all
    ORDER BY ClientID, DateTime
    LIMIT 10
    FORMAT TabSeparatedWithNames
'''

get_clickhouse_df(q)

,ClientID,DateTime,StartURL,Params
0,0,2016-10-01 06:12:13,https://yandex.ru/support/metrika/troubleshooting.xml,[]
1,0,2016-10-01 18:15:13,https://yandex.ru/support/metrika/troubleshooting.xml,[]
2,0,2016-10-01 19:11:30,https://yandex.ru/support/metrika/troubleshooting.xml,[]
3,0,2016-10-02 11:32:10,https://yandex.ru/support/metrika/objects/reachgoal.xml,[]
4,0,2016-10-03 15:35:56,https://yandex.ru/support/metrika/objects/reachgoal.xml,[]
5,0,2016-10-03 21:24:32,https://yandex.ru/support/metrika/webvisor/about-webvisor.xml,[]
6,0,2016-10-04 01:28:11,https://yandex.ru/support/metrika/webvisor/about-webvisor.xml,[]
7,0,2016-10-04 07:45:06,https://yandex.ru/support/metrika/webvisor/about-webvisor.xml,[]
8,0,2016-10-04 09:25:04,https://yandex.ru/support/metrika/general/glossary.xml,[]
9,0,2016-10-04 09:25:05,https://yandex.ru/support/metrika/general/glossary.xml,[]


In [38]:
q = '''
    SELECT 
        ClientID,
        DateTime,
        StartURL, 
        Params
    FROM visits_all
    WHERE (ClientID != 0) AND empty(Params)
    ORDER BY ClientID, DateTime
    LIMIT 10
    FORMAT TabSeparatedWithNames
'''

get_clickhouse_df(q)

,ClientID,DateTime,StartURL,Params
0,1480711580639,2017-01-21 19:06:42,https://yandex.ru/support/metrika/troubleshooting.xml,[]
1,14778924825277,2016-10-31 08:43:38,https://yandex.ru/support/metrika/,[]
2,14800770644117,2016-12-03 07:41:10,https://yandex.ru/support/metrika/data/e-commerce.xml,[]
3,14865462604714,2017-02-25 00:07:51,https://yandex.ru/support/metrika/general/glossary.xml,[]
4,14879840339434,2017-02-25 03:58:54,https://yandex.ru/support/metrika/data/visit-params.xml,[]
5,14921126236380,2017-05-03 22:22:47,https://yandex.ru/support/metrika/visitors/loyalty.xml,[]
6,14951812245812,2017-07-01 19:37:52,https://forms.yandex.ru/surveys/1705/?iframe=1&theme=support&title=Начало работы&lang=ru&path=/metrika/quick-start.xml&lang=ru&referer=https:%2F%2Fwww.google.ru%2F&url=https:%2F%2Fyandex.ru%2Fsupport%2Fmetrika%2Fquick-start.xml&feedback_sid=3342194641495035808&mobile=no&fromhost=yandex.ru&service=metrika&login=centerchs&owner=yes&base=/support,[]
7,15053924165319,2017-09-14 15:33:56,https://yandex.ru/support/metrika/general/access.xml,[]
8,146236954633384,2017-04-10 16:18:39,https://yandex.ru/support/metrika/general/regexp.xml,[]
9,146455313248369,2017-05-18 07:44:07,https://yandex.ru/support/metrika/code/counter-initialize.xml,[]


Как и в классическом SQL для того, чтобы получить только уникальные различающиеся строки, нужно использовать ключевое слово `DISTINCT`.

Посмотрим на то, какие есть значение `Params`.

In [39]:
q = '''
    SELECT
        DISTINCT Params
    FROM visits_all
    LIMIT 20
    FORMAT TabSeparatedWithNames
'''

get_clickhouse_df(q)

,Params
0,"['{""login"":true}','{""login"":true}']"
1,[]
2,"['{""login"":true}']"
3,"['{""login"":false}']"
4,"['{""login"":false}','{""login"":false}']"
5,"['{""login"":true}','{""login"":true}','{""login"":true}']"
6,"['{""login"":true}','{""login"":true}','{""login"":true}','{""login"":true}']"
7,"['{""login"":false}','{""login"":false}','{""login"":false}','{""login"":false}']"
8,"['{""login"":false}','{""login"":false}','{""login"":false}']"
9,"['{""login"":true}','{""login"":true}','{""login"":true}','{""login"":true}','{""login"":true}']"


## Аггрегатные функции

Перейдем к агрегатным функциям и посчитаем число сессий, просмотров и пользователей.
`count` использует для подсчета числа строк, `sum` - для суммы, `uniq` - для числа уникальных значений.
Стоит помнить, что по default'у `uniq` выдает приближенную оценку, но есть и функция для точного подсчета `uniqExact`.

In [40]:
q = '''
    SELECT
        count() as visits,
        sum(PageViews) as hits,
        uniq(ClientID) as users
    FROM visits_all
    FORMAT TabSeparatedWithNames
'''

get_clickhouse_df(q)

,visits,hits,users
0,1222286,3293781,620355


Посчитаем все те же показатели для разных браузеров.
Если указать `WITH TOTALS`, то БД также посчитает суммарные значения.

In [41]:
q = '''
    SELECT
        count() as visits,
        sum(PageViews) as hits,
        uniq(ClientID) as users,
        Browser as browser
    FROM visits_all
    GROUP BY browser
    WITH TOTALS
    ORDER BY users DESC
    LIMIT 10
    FORMAT TabSeparatedWithNames
'''

get_clickhouse_df(q)

,visits,hits,users,browser
0,689755,1878324,343790,chrome
1,145838,405446,76274,firefox
2,169208,478621,73004,yandex_browser
3,79165,196500,37949,opera
4,30391,70906,21679,chromemobile
5,24467,50672,17839,safari_mobile
6,33617,90466,16835,safari
7,12048,31748,9216,msie
8,10987,28921,7046,edge
9,5971,12778,4432,android_browser


Отфильтруем браузеры с очень маленькой аудиторией, добавив фильтр в секцию `HAVING`. В `HAVING` указываются фильтрации на значение агрегатных функций.

In [42]:
q = '''
    SELECT
        count() as visits,
        sum(PageViews) as hits,
        uniq(ClientID) as users,
        Browser as browser
    FROM visits_all
    GROUP BY browser
    HAVING users >= 10000
    ORDER BY users DESC
    FORMAT TabSeparatedWithNames
'''

get_clickhouse_df(q)

,visits,hits,users,browser
0,689755,1878324,343790,chrome
1,145838,405446,76274,firefox
2,169208,478621,73004,yandex_browser
3,79165,196500,37949,opera
4,30391,70906,21679,chromemobile
5,24467,50672,17839,safari_mobile
6,33617,90466,16835,safari


В ряде случаев очень полезны бывают агрегатные функции `argMin` и `argMax`, которые позволяют получить аргумент, который соответствует минимальному или максимальному значению функции.
Для примера посмотрим с каких источников трафика приходили впервые клиенты на наш сайт.

In [43]:
q = '''
    SELECT 
        ClientID,
        argMin(LastTrafficSource, DateTime) as TraficSource,
        min(DateTime) as Time
    FROM visits_all SAMPLE 1/10
    GROUP BY ClientID 
    LIMIT 10
    FORMAT TabSeparatedWithNames
'''

get_clickhouse_df(q)

,ClientID,TraficSource,Time
0,1492082334667013933,organic,2017-06-15 18:19:42
1,1472637049942465862,referral,2016-10-02 21:37:09
2,1479801973409171068,organic,2016-12-19 08:31:45
3,1504627014257703686,direct,2017-09-07 13:28:18
4,1476794766215335835,referral,2016-10-18 17:06:25
5,1472227461958972179,referral,2016-12-22 18:38:33
6,1485823812347745446,organic,2017-02-15 15:31:05
7,1476954727667706155,organic,2017-01-24 15:42:37
8,1481930425910644568,referral,2017-01-10 19:22:56
9,1497376107135577420,organic,2017-06-17 21:18:14


### Модификатор агрегатных функций -If

К любой агрегатной функции можно добавить суффикс -If и тогда у функции добавится еще один параметр - условие и при подсчете значения будут учитываться только те строки, которые удовлетворяют условию.

Давайте посчитаем долю мобильных устройств в разных странах по пользователям и трафику.

In [44]:
q = '''
    SELECT
        count() as total_visits,
        DeviceCategory as device
    FROM visits_all
    GROUP BY device
    ORDER BY total_visits DESC
    FORMAT TabSeparatedWithNames
'''

get_clickhouse_df(q)

,total_visits,device
0,1142837,1
1,64845,2
2,14527,3
3,77,4


In [45]:
q = '''
    SELECT
        count() as total_visits,
        countIf(DeviceCategory IN ('2', '1')) as mobile_visits,
        uniq(ClientID) as total_users,
        uniqIf(ClientID, DeviceCategory IN ('1', '2')) as mobile_users,
        round(100*mobile_visits/total_visits, 2) as mobile_visits_share,
        round(100*mobile_users/total_users, 2) as mobile_users_share,
        RegionCountry as country
    FROM visits_all
    GROUP BY country
    ORDER BY total_visits DESC
    LIMIT 10
    FORMAT TabSeparatedWithNames
'''

get_clickhouse_df(q)

,total_visits,mobile_visits,total_users,mobile_users,mobile_visits_share,mobile_users_share,country
0,978075,965574,507588,499661,98.72,98.44,Russia
1,90897,90042,48006,47480,99.06,98.90,Ukraine
2,51159,50716,24180,23894,99.13,98.82,Belarus
3,43557,43537,4045,4040,99.95,99.88,NaN
4,13852,13768,8580,8506,99.39,99.14,Kazakhstan
5,3643,3566,2791,2725,97.89,97.64,Turkey
6,3515,3473,2178,2140,98.81,98.26,Germany
7,3308,3264,2295,2253,98.67,98.17,United States
8,2995,2923,1897,1872,97.60,98.68,Moldova
9,1995,1983,1328,1321,99.40,99.47,Netherlands


## Семплирование

In [46]:
%%time
q = '''
    SELECT 
        uniq(ClientID) as users,
        Date
    FROM visits_all
    GROUP BY Date
    ORDER BY Date
    FORMAT TabSeparatedWithNames
'''

df1 = get_clickhouse_df(q)

CPU times: user 5.04 ms, sys: 1.48 ms, total: 6.52 ms
Wall time: 201 ms


In [47]:
%%time
q = '''
    SELECT 
        10*uniq(ClientID) as users,
        Date
    FROM visits_all SAMPLE 1/10
    GROUP BY Date
    ORDER BY Date
    FORMAT TabSeparatedWithNames
'''

df2 = get_clickhouse_df(q)

CPU times: user 5.78 ms, sys: 1.47 ms, total: 7.24 ms
Wall time: 87.1 ms


In [48]:
df1.set_index('Date').join(df2.set_index('Date'), rsuffix = '_sample').head(20)

,users,users_sample
Date,,
2016-10-01,1482,1480
2016-10-02,1531,1300
2016-10-03,3652,3600
2016-10-04,4220,4160
2016-10-05,4217,4280
2016-10-06,4069,4000
2016-10-07,3596,3660
2016-10-08,1590,1560
2016-10-09,1639,1720


In [49]:
def load_users_iter(i, N = 10):
    q = '''
        SELECT 
            uniq(ClientID) as users,
            Date
        FROM visits_all SAMPLE 1/{N} OFFSET {i}/{N}
        GROUP BY Date
        ORDER BY Date
        FORMAT TabSeparatedWithNames
    '''.format(i = i, N = N)
    return get_clickhouse_df(q)

tmp_dfs = []
for i in range(10):
    tmp_dfs.append(load_users_iter(i))

In [50]:
df3 = pd.concat(tmp_dfs).groupby('Date').sum()

In [51]:
df3.head()

,users
Date,
2016-10-01,1482
2016-10-02,1531
2016-10-03,3652
2016-10-04,4220
2016-10-05,4217


## Подзапросы

Также в ClickHouse есть функции для расчета процентилей `quantile(level)(x)`. Давайте посчитаем распределение числа визитов на посетителя за месяц.

Заодно познакомимся с подзапросами. Для начала расчитаем число сессий на пользователя в сентябре.

In [52]:
q = '''
    SELECT
        count() as user_visits,
        ClientID
    FROM visits_all
    WHERE Date >= '2017-09-01' 
        AND Date <= '2017-09-31'
        AND ClientID != 0
    GROUP BY ClientID
    LIMIT 10
    FORMAT TabSeparatedWithNames
'''

get_clickhouse_df(q)

,user_visits,ClientID
0,1,1502378597495977022
1,1,15052186251004790059
2,6,1504627014257703686
3,1,1490610856982761788
4,1,1503923400410463209
5,1,1504179477936790080
6,2,1472623288822470147
7,1,1500634310192225419
8,1,150524544255174183
9,1,1506170539102569045


In [53]:
q = '''
    SELECT
        quantile(0.25)(user_visits) as user_visits_p25,
        quantile(0.5)(user_visits) as user_visits_p50,
        quantile(0.75)(user_visits) as user_visits_p75,
        quantile(0.9)(user_visits) as user_visits_p90,
        quantile(0.95)(user_visits) as user_visits_p95,
        quantile(0.99)(user_visits) as user_visits_p99
    FROM
        (SELECT
            count() as user_visits,
            ClientID
        FROM visits_all
        WHERE Date >= '2017-09-01' 
            AND Date <= '2017-09-31'
            AND ClientID != 0
        GROUP BY ClientID)
    FORMAT TabSeparatedWithNames
'''

get_clickhouse_df(q)

,user_visits_p25,user_visits_p50,user_visits_p75,user_visits_p90,user_visits_p95,user_visits_p99
0,1,1,1,3,4,9


Создадим таблицу с посетителями, которые заходили на сайт с поиска Яндекса.

In [54]:
q = '''
    CREATE TABLE ya_ids ENGINE = Log AS 
        SELECT DISTINCT
            ClientID
        FROM visits_all
        WHERE domain(Referer) LIKE '%yandex.%'
'''

get_clickhouse_data(q)

''

In [55]:
q = 'SELECT count() from ya_ids'
print(get_clickhouse_data(q))

307110



In [57]:
q = '''
    SELECT
        round(count()/uniq(ClientID), 2) as avg_visits_per_user
    FROM visits_all
    WHERE ClientID IN (SELECT ClientID FROM ya_ids)
    FORMAT TabSeparatedWithNames
'''

get_clickhouse_df(q)

,avg_visits_per_user
0,2.49


In [59]:
q = '''
    SELECT 
        round(count()/uniq(ClientID), 2) as avg_visits_per_user,
        if(ClientID IN (SELECT ClientID FROM ya_ids), 'yandex', 'no yandex') as from_yandex
    FROM visits_all
    GROUP BY from_yandex
    FORMAT TabSeparatedWithNames
'''

get_clickhouse_df(q)

,avg_visits_per_user,from_yandex
0,2.49,yandex
1,1.46,no yandex


## Полезные функции
### Функции для работы с датами

В ClickHouse есть удобные функции по работе с датой и временем, например, чтобы получить текущие дату и время. 
Также для дат и времени поддержаны операции сложения и вычитания, которые прибавляют/отнимают соответственно дни и секунды.

In [37]:
q = '''
    SELECT
        today() as today,
        yesterday() as yesterday,
        today() - 7 as previous_week
    FORMAT TabSeparatedWithNames
'''
get_clickhouse_df(q)

,today,yesterday,previous_week
0,2017-11-06,2017-11-05,2017-10-30


In [38]:
q = '''
    SELECT
        now() as now,
        now() - 60*60 as prev_hour,
        now() - 24*60*60 as previous_day
    FORMAT TabSeparatedWithNames
'''
get_clickhouse_df(q)

,now,prev_hour,previous_day
0,2017-11-06 06:39:15,2017-11-06 05:39:15,2017-11-05 06:39:15


In [39]:
q = '''
    SELECT
        toYear(now()) as year,
        toMonth(now()) as month,
        toDayOfMonth(now()) as day,
        toHour(now()) as hour,
        toMinute(now()) as minutes
    FORMAT TabSeparatedWithNames
'''

get_clickhouse_df(q)

,year,month,day,hour,minutes
0,2017,11,6,6,39


Можем посчитать распределение заходов на сайт по дням недели.

In [44]:
q = '''
    SELECT
        count() as visits,
        toDayOfWeek(Date) as day
    FROM visits_all
    GROUP BY day
    FORMAT TabSeparatedWithNames
'''

plotly_bar_df(get_clickhouse_df(q).set_index('day'), title = 'Visits by day of week')

Можно сделать ответ еще лучше, заменив id дней недели на их названия.

In [45]:
q = '''
    SELECT
        count() as visits,
        transform(
            toDayOfWeek(Date),
            [1, 2, 3, 4, 5, 6, 7],
            ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday'],
            'Error'
        ) as day
    FROM visits_all
    GROUP BY day
    FORMAT TabSeparatedWithNames
'''

plotly_bar_df(
    get_clickhouse_df(q).set_index('day'), 
    title = 'Visits by day of week', 
    index = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday'])

Еще одна полезная функция по работе с датами - это приведение к началу периода `toStartOfMonth`, `toStartOfYear`, `toStartOfMinute`, `toStartOfHour` и т.д.

In [46]:
q = '''
    SELECT
        count() as visits,
        toStartOfMonth(Date) as date
    FROM visits_all
    WHERE RegionCountry = 'Russia'
    GROUP BY date
    FORMAT TabSeparatedWithNames
'''

plotly_line_df(get_clickhouse_df(q).set_index('date'), title = 'Visits from Russia')

In [47]:
q = '''
    SELECT
        count() as visits,
        toStartOfFiveMinute(DateTime) as date
    FROM visits_all
    WHERE RegionCountry = 'Russia'
        AND Date = '2017-09-01'
    GROUP BY date
    ORDER BY date
    FORMAT TabSeparatedWithNames
'''

plotly_line_df(get_clickhouse_df(q).set_index('date'), title = 'Visits from Russia')

In [48]:
q = '''
    SELECT
        countIf(Date = '2017-09-01') as today_visits,
        countIf(Date = toDate('2017-09-01') - 1) as yesterday_visits,
        countIf(Date = toDate('2017-09-01') - 7) as prev_week_visits,
        toStartOfFiveMinute(DateTime) as date
    FROM visits_all
    WHERE RegionCountry = 'Russia'
        AND Date IN ('2017-09-01', toDate('2017-09-01') - 1, toDate('2017-09-01') - 7)
    GROUP BY date
    ORDER BY date
    FORMAT TabSeparatedWithNames
'''

plotly_line_df(get_clickhouse_df(q).set_index('date'), title = 'Visits from Russia')

### Функции по работе со строками и URL

In [49]:
q = '''
    SELECT
        count() as visits,
        Referer
    FROM visits_all
    GROUP BY Referer
    ORDER BY visits DESC
    LIMIT 10
    FORMAT TabSeparatedWithNames
'''

get_clickhouse_df(q)

,visits,Referer
0,270341,NaN
1,215959,https://google.ru/
2,32354,https://google.com.ua/
3,16399,https://metrika.yandex.ru/list
4,15161,https://yandex.ru/support/
5,15109,https://google.by/
6,13561,https://google.com/
7,10407,https://metrika.yandex.ru/calls/
8,7482,https://yandex.ru/support/metrika/reports/add-...
9,5973,https://yandex.ru/support/direct/statistics/me...


In [50]:
q = '''
    SELECT
        count() as visits,
        Referer
    FROM visits_all
    WHERE Referer NOT LIKE '%yandex.%'
    GROUP BY Referer
    ORDER BY visits DESC
    LIMIT 10
    FORMAT TabSeparatedWithNames
'''

get_clickhouse_df(q)

,visits,Referer
0,270341,NaN
1,215959,https://google.ru/
2,32354,https://google.com.ua/
3,15109,https://google.by/
4,13561,https://google.com/
5,4046,https://google.kz/
6,2935,https://away.vk.com/away.php
7,1661,http://marketing-wiki.ru/wiki/Разметка_ссылок_...
8,1264,https://google.ru
9,1119,https://vk.com/away.php


In [60]:
q = r'''
    SELECT
        count() as visits,
        Referer
    FROM visits_all
    WHERE match(Referer, 'https?://google.(ru|com)')
    GROUP BY Referer
    ORDER BY visits DESC
    LIMIT 10
    FORMAT TabSeparatedWithNames
'''

get_clickhouse_df(q)

,visits,Referer
0,215959,https://google.ru/
1,32354,https://google.com.ua/
2,13561,https://google.com/
3,1264,https://google.ru
4,1008,http://google.ru/
5,265,https://google.com.tr/
6,215,https://google.com.ua
7,124,https://google.com
8,105,https://google.com.ph/
9,98,https://google.com.vn/


In [51]:
q = '''
    SELECT
        count() as visits,
        Referer,
        domain(Referer) as RefererDomain,
        cutToFirstSignificantSubdomain(Referer) as RefererSignSubdomain
    FROM visits_all
    WHERE Referer NOT LIKE '%yandex.%'
    GROUP BY Referer
    ORDER BY visits DESC
    LIMIT 10
    FORMAT TabSeparatedWithNames
'''

get_clickhouse_df(q)

,visits,Referer,RefererDomain,RefererSignSubdomain
0,270341,NaN,NaN,NaN
1,215959,https://google.ru/,google.ru,google.ru
2,32354,https://google.com.ua/,google.com.ua,google.com.ua
3,15109,https://google.by/,google.by,google.by
4,13561,https://google.com/,google.com,google.com
5,4046,https://google.kz/,google.kz,google.kz
6,2935,https://away.vk.com/away.php,away.vk.com,vk.com
7,1661,http://marketing-wiki.ru/wiki/Разметка_ссылок_...,marketing-wiki.ru,marketing-wiki.ru
8,1264,https://google.ru,google.ru,google.ru
9,1119,https://vk.com/away.php,vk.com,vk.com


In [52]:
q = '''
    SELECT
        count() as visits,
        Referer
    FROM visits_all
    WHERE Referer LIKE '%mail.ru/search%'
    GROUP BY Referer
    ORDER BY visits DESC
    LIMIT 10
    FORMAT TabSeparatedWithNames
'''

get_clickhouse_df(q)

,visits,Referer
0,27,http://go.mail.ru/search?fm=1&q=как дать досту...
1,1,http://go.mail.ru/search?fr=ffxtn107&gp=802821...
2,1,http://go.mail.ru/search?fr=ffvbm1.0.0.51&fr2=...
3,1,http://go.mail.ru/search?fr=chxtn12.0.12&q=уст...
4,1,http://go.mail.ru/search?q=что это за адрес ht...
5,1,http://go.mail.ru/search?fr=chvbm7.0.30&fr2=qu...


In [53]:
q = '''
    SELECT
        count() as visits,
        extractURLParameter(Referer, 'q') as query
    FROM visits_all
    WHERE Referer LIKE '%mail.ru/search%'
    GROUP BY query
    ORDER BY visits DESC
    LIMIT 10
    FORMAT TabSeparatedWithNames
'''

get_clickhouse_df(q)

,visits,query
0,27,как дать доступ в яндекс метрику
1,1,https:%2F%2Fmetrika.yandex.ru%2Fdashboard%3Fid...
2,1,что это за адрес https mc yandex ru metrika wa...
3,1,яндекс метрик%5C
4,1,установить на сайт счетчик яндекс
5,1,яндексметрика


In [54]:
q = '''
    SELECT
        count() as visits,
        StartURL,
        path(StartURL) as path
    FROM visits_all
    GROUP BY StartURL
    ORDER BY visits DESC
    LIMIT 10
    FORMAT TabSeparatedWithNames
'''

get_clickhouse_df(q)

,visits,StartURL,path
0,133394,https://yandex.ru/support/metrika/,/support/metrika/
1,91121,https://yandex.ru/support/metrika/reports/add-...,/support/metrika/reports/add-goals.xml
2,66650,https://yandex.ru/support/metrika/general/acce...,/support/metrika/general/access.xml
3,55021,https://yandex.ru/support/metrika/objects/reac...,/support/metrika/objects/reachgoal.xml
4,51122,https://yandex.ru/support/metrika/quick-start.xml,/support/metrika/quick-start.xml
5,48977,https://yandex.ru/support/metrika/general/glos...,/support/metrika/general/glossary.xml
6,45955,https://yandex.ru/support/metrika/troubleshoot...,/support/metrika/troubleshooting.xml
7,44190,https://yandex.ru/support/metrika/general/goal...,/support/metrika/general/goals.xml
8,34018,https://yandex.ru/support/metrika/data/e-comme...,/support/metrika/data/e-commerce.xml
9,32447,https://yandex.ru/support/metrika/general/targ...,/support/metrika/general/target-call.xml
